![](https://miro.medium.com/max/1400/1*gYe2FMr9lKys2Wmo1v-s7A.jpeg)

# 1. H&M Personalized Fashion Recommendations

## Problem Statement: Given the features and image of a product,finding the similar products for recommendations using weighted nearest neighbors.
### 1.1.1 we will be using features and image of the product.
### I will use only 10 points, then you can scale to over 100k

### 1.1.2 Text featurization. : IDF weighted word2vec of Title

In [ ]:
#importing required libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from collections import Counter
import os.path
from sklearn.metrics import pairwise_distances
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout
plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

In [ ]:
#improting data as a pandas dataframe
HM_data = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
HM_data=HM_data.head(10000)

In [ ]:
#getting the shape of the data
print("Number of data-points in the data:" , HM_data.shape[0])
print("Number of features in the data :" , HM_data.shape[1])

In [ ]:
#getting the column names
print(HM_data.columns)

In [ ]:
# Getting product image path
images_path=[]
i=0
for p in HM_data['article_id'].tolist():
    path= '../input/h-and-m-personalized-fashion-recommendations/images/' + '0' + str(p)[:2] + '/'+ '0' + str(p) +'.jpg'

    if os.path.exists(path):
        i+=1
        images_path.append(path)
    else: images_path.append(None)
print(f'There is {i} article with corresponding image')
    

In [ ]:
#taking only 6 features and 7th feature will be image url
HM_data['image_path']=images_path
HM_data = HM_data[['article_id', 'prod_name', 'colour_group_name', 'image_path',
             'product_type_name', 'product_group_name', 'detail_desc']]
#getting the shape of the data
print("Number of data-points in the data:" , HM_data.shape[0])
print("Number of features in the data :" , HM_data.shape[1])

In [ ]:
#getting the head of the data
HM_data.head(3)

## 2.1 Missing data for various features.
### 2.1.1 Basic stats for the feature: product_type_name

In [ ]:
print(HM_data['product_type_name'].describe())

## Observations:

* About 12% (11169/105542) of the products are Trousers.
* There are 131 unique product types.

In [ ]:
#find the 10 most frequent product_type_names.
product_type_count = Counter(list(HM_data['product_type_name']))
product_type_count.most_common(10)

### 2.1.2 Basic stats for the feature: color

In [ ]:
print(HM_data['colour_group_name'].describe())

## Observations:

* we have 50 unique colors
* Over 20% of products are black in color

In [ ]:
#find the 10 most frequent colors.
color_count = Counter(list(HM_data['colour_group_name']))
color_count.most_common(10)

### 2.1.3 Basic stats for the feature: detail_desc

In [ ]:
print(HM_data['detail_desc'].describe())

## Observations

* Most of the products have a detail_desc.
* detail_desc are fairly descriptive of what the product is.
* We use detail_desc extensively in this notebook as they are short and informative.

## 2.2 Remove near duplicate items using detail_desc feature

In [ ]:
print("Number of duplicates present in articles: {}".format(sum(HM_data.duplicated("detail_desc"))))

In [ ]:
HM_data

In [ ]:
#removing data-points with short-titles
HM_data_sorted = HM_data[HM_data['detail_desc'].apply(lambda x: len(str(x).split())>4)]
print("After removal of products with short description:", HM_data_sorted.shape[0])

In [ ]:
# Sort the whole data based on title (alphabetical order of detail_desc) 
HM_data_sorted.sort_values('detail_desc',inplace=True, ascending=False)

### 2.2.1 Removing duplicates with Text sorted

In [ ]:
#getting the duplicates 
indices = []
for i,row in HM_data_sorted.iterrows():
    indices.append(i)
    
import itertools
stage1_dedupe_asins = []
i = 0
j = 0
num_data_points = HM_data_sorted.shape[0]
while i < num_data_points and j < num_data_points:
       
    previous_i = i
    a = HM_data['detail_desc'].loc[indices[i]].split()

    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:
        b = HM_data['detail_desc'].loc[indices[j]].split()

        # store the maximum length of two strings
        length = max(len(a), len(b))

        #count is used to store the number of words that are matched in both strings
        count  = 0

        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        # if the number of words in which both strings differ are > 2 , we are considering it as those two apperals are different
        # if the number of words in which both strings differ are < 2 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words we include the 1st string index
            stage1_dedupe_asins.append(HM_data_sorted['article_id'].loc[indices[i]])

            # if the comaprision between is between num_data_points, num_data_points-1 strings and they differ in more than 2 words we include both
            if j == num_data_points-1: stage1_dedupe_asins.append(HM_data_sorted['article_id'].loc[indices[j]])

            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break    

In [ ]:
#removing duplicated
HM_data = HM_data.loc[HM_data['article_id'].isin(stage1_dedupe_asins)]

In [ ]:
#getting the number of data-points after removing the duplicates
print('Number of data points : ', HM_data.shape[0])

### 2.2.2 Removing duplicates which are not similar adjacently but very similar

In [ ]:
indices = []
for i,row in HM_data.iterrows():
    indices.append(i)

stage2_dedupe_asins = []
while len(indices)!=0:
    i = indices.pop()
    stage2_dedupe_asins.append(HM_data['article_id'].loc[i])
    # consider the first apperal's title
    a = HM_data['detail_desc'].loc[i].split()
 
    for j in indices:
        
        b = HM_data['detail_desc'].loc[j].split()
        length = max(len(a),len(b))
        
        # count is used to store the number of words that are matched in both strings
        count  = 0

        for k in itertools.zip_longest(a,b): 
            if (k[0]==k[1]):
                count += 1

        #if the number of words in which both strings differ are < 3 ,
        #we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) < 3:
            indices.remove(j)

In [ ]:
#removing duplicate further
HM_data = HM_data.loc[HM_data['article_id'].isin(stage2_dedupe_asins)]

## 3. Text-PreProcessing:

In [ ]:
#getting the number of data-points after removing all the duplicates
print('Number of data points after stage two of dedupe: ',HM_data.shape[0])

In [ ]:
HM_data

In [ ]:
# removing stop-words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalnum()))
            # Convert all letters to lower-case
            word = word.lower()
            # stop-word removal
            if not word in stop_words:
                string += word + " "
        HM_data[column][index] = string
        
        
# we take each title and we text-preprocess it.
for index, row in HM_data.iterrows():
    nlp_preprocessing(row['detail_desc'], index, 'detail_desc') 

# Stemming

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
print(stemmer.stem('arguing'))
print(stemmer.stem('fishing'))
# stemming doesn't convert the tokens into meaningful words 

In [ ]:
# function to display an image
def display_img(url,ax,fig):
    # we get the url of the apparel and download it
    #response = requests.get(url)
    img = Image.open(url)
    # we will display it in notebook 
    plt.imshow(img)

# function to generate the heatmap
def plot_heatmap(keys, values, labels, url, text):
        # keys: list of words of recommended title
        # values: len(values) ==  len(keys), values(i) represents the occurence of the word keys(i)
        # labels: len(labels) == len(keys), the values of labels depends on the model we are using
                # if model == 'bag of words': labels(i) = values(i)
                # if model == 'tfidf weighted bag of words':labels(i) = tfidf(keys(i))
                # if model == 'idf weighted bag of words':labels(i) = idf(keys(i))
        # url : apparel's url
        # we will devide the whole figure into two parts
        gs = gridspec.GridSpec(2, 2, width_ratios=[4,1], height_ratios=[4,1]) 
        fig = plt.figure(figsize=(25,3))
        # 1st, ploting heat map that represents the count of commonly ocurred words in title2
        ax = plt.subplot(gs[0])
        # it displays a cell in white color if the word is intersection(list of words of title1 and list of words of title2), in black if not
        ax = sns.heatmap(np.array([values]), annot=np.array([labels]))
        ax.set_xticklabels(keys) # set that axis labels as the words of title
        ax.set_title(text) # apparel title        
        # 2nd, plotting image of the the apparel
        ax = plt.subplot(gs[1])
        # we don't want any grid lines for image and no labels on x-axis and y-axis
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
        # we call dispaly_img based with paramete url
        display_img(url, ax, fig)       
        # displays combine figure ( heat map and image together)
        plt.show()

# function to display heatmap and image        
def plot_heatmap_image(doc_id, vec1, vec2, url, text, model):
    # doc_id : index of the title1
    # vec1 : input apparels's vector, it is of a dict type {word:count}
    # vec2 : recommended apparels's vector, it is of a dict type {word:count}
    # url : apparels image url
    # text: title of recomonded apparel (used to keep title of image)
    # model, it can be any of the models, 
        # 1. bag_of_words
        # 2. tfidf
        # 3. idf
    # we find the common words in both titles, because these only words contribute to the distance between two title vec's
    intersection = set(vec1.keys()) & set(vec2.keys()) 
    # we set the values of non intersecting words to zero, this is just to show the difference in heatmap
    for i in vec2:
        if i not in intersection:
            vec2[i]=0
    # for labeling heatmap, keys contains list of all words in title2
    keys = list(vec2.keys())
    #  if ith word in intersection(lis of words of title1 and list of words of title2): values(i)=count of that word in title2 else values(i)=0 
    values = [vec2[x] for x in vec2.keys()]  
    # labels: len(labels) == len(keys), the values of labels depends on the model we are using
        # if model == 'bag of words': labels(i) = values(i)
        # if model == 'tfidf weighted bag of words':labels(i) = tfidf(keys(i))
        # if model == 'idf weighted bag of words':labels(i) = idf(keys(i))
    if model == 'bag_of_words':
        labels = values
    elif model == 'tfidf':
        labels = []
        for x in vec2.keys():
            # tfidf_title_vectorizer.vocabulary_ it contains all the words in the corpus
            # tfidf_title_features[doc_id, index_of_word_in_corpus] will give the tfidf value of word in given document (doc_id)
            if x in  tfidf_title_vectorizer.vocabulary_:
                labels.append(tfidf_title_features[doc_id, tfidf_title_vectorizer.vocabulary_[x]])
            else:
                labels.append(0)
    elif model == 'idf':
        labels = []
        for x in vec2.keys():
            # idf_title_vectorizer.vocabulary_ it contains all the words in the corpus
            # idf_title_features[doc_id, index_of_word_in_corpus] will give the idf value of word in given document (doc_id)
            if x in  idf_title_vectorizer.vocabulary_:
                labels.append(idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[x]])
            else:
                labels.append(0)
    plot_heatmap(keys, values, labels, url, text)

#function to convert text into vector
def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    # words stores list of all words in given string, you can try 'words = text.split()' this will also gives same result
    return Counter(words) # Counter counts the occurence of each word in list, it returns dict type object {word1:count}

# function to display the result
def get_result(doc_id, content_a, content_b, url, model):
    text1 = content_a
    text2 = content_b
    # vector1 = dict{word11:#count, word12:#count, etc.}
    vector1 = text_to_vector(text1)
    # vector2 = dict{word21:#count, word22:#count, etc.}
    vector2 = text_to_vector(text2)
    plot_heatmap_image(doc_id, vector1, vector2, url, text2, model)

# 5.1 Bag of Words (BoW) on product detail_desc

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer = CountVectorizer()
title_features   = title_vectorizer.fit_transform(HM_data['detail_desc'])
title_features.get_shape()
# title_features[doc_id, index_of_word_in_corpus] = number of times the word occured in that doc

In [ ]:
data=HM_data.copy()
def bag_of_words_model(doc_id, num_results):
    # doc_id: apparel's id in given corpus
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    pairwise_dist = pairwise_distances(title_features,title_features[doc_id])
    # np.argsort will return indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
    #data frame indices of the 9 smallest distace's
    df_indices = list(data.index[indices])    
    for i in range(0,len(indices)):
        # we will pass 1. doc_id, 2. title1, 3. title2, url, model
        get_result(indices[i],data['detail_desc'].loc[df_indices[0]], data['detail_desc'].loc[df_indices[i]], data['image_path'].loc[df_indices[i]], 'bag_of_words')
        print('article_id :',data['article_id'].loc[df_indices[i]])
        print ('detail_desc:', data['detail_desc'].loc[df_indices[i]])
        print ('Euclidean similarity with the query image :', pdists[i])
        print('='*60)
#call the bag-of-words model for a product to get similar products
bag_of_words_model(1582, 20)
# in the output heat map each value represents the tfidf values of the label word, the color represents the intersection with inputs title


# TF-IDF based product similarity

In [ ]:
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_title_features = tfidf_title_vectorizer.fit_transform(data['detail_desc'])

In [ ]:
def tfidf_model(doc_id, num_results):
    # doc_id: apparel's id in given corpus  
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    pairwise_dist = pairwise_distances(tfidf_title_features,tfidf_title_features[doc_id])
    # np.argsort will return indices of 9 smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the 9 smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
    #data frame indices of the 9 smallest distace's
    df_indices = list(data.index[indices])
    for i in range(0,len(indices)):
        # we will pass 1. doc_id, 2. title1, 3. title2, url, model
        get_result(indices[i], data['detail_desc'].loc[df_indices[0]], data['detail_desc'].loc[df_indices[i]], data['image_path'].loc[df_indices[i]], 'tfidf')
        print('article_id :',data['article_id'].loc[df_indices[i]])
        print ('Eucliden distance from the given image :', pdists[i])
        print('='*125)
tfidf_model(1582, 20)
# in the output heat map each value represents the tfidf values of the label word, the color represents the intersection with inputs title

# IDF based product similarity

In [ ]:
idf_title_vectorizer = CountVectorizer()
idf_title_features = idf_title_vectorizer.fit_transform(data['detail_desc'])

In [ ]:
def n_containing(word):
    # return the number of documents which had the given word
    return sum(1 for blob in data['detail_desc'] if word in blob.split())
def idf(word):
    # idf = log(number of docs / number of docs which had the given word)
    return math.log(data.shape[0] / (n_containing(word)))

In [ ]:
# converting the values into float
idf_title_features  = idf_title_features.astype(np.float)
for i in idf_title_vectorizer.vocabulary_.keys():
    # for every word in whole corpus we will find its idf value
    idf_val = idf(i)
    # to calculate idf_title_features we will replace the count values with the idf values of the word
    # idf_title_features[:, idf_title_vectorizer.vocabulary_[i]].nonzero()[0] will return all documents in which the word i is present
    for j in idf_title_features[:, idf_title_vectorizer.vocabulary_[i]].nonzero()[0]:        
        # we will replace the count values of word i in document j with idf_value of word i 
        # idf_title_features[doc_id, index_of_word_in_courpus] = idf value of word
        idf_title_features[j,idf_title_vectorizer.vocabulary_[i]] = idf_val  

In [ ]:
def idf_model(doc_id, num_results):
    # doc_id: apparel's id in given corpus    
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    pairwise_dist = pairwise_distances(idf_title_features,idf_title_features[doc_id])
    # np.argsort will return indices of 9 smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the 9 smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
    #data frame indices of the 9 smallest distace's
    df_indices = list(data.index[indices])
    for i in range(0,len(indices)):
        get_result(indices[i],data['detail_desc'].loc[df_indices[0]], data['detail_desc'].loc[df_indices[i]], data['image_path'].loc[df_indices[i]], 'idf')
        print('article_id :',data['article_id'].loc[df_indices[i]])
        print ('euclidean distance from the given image :', pdists[i])
        print('='*125)        
idf_model(1582,20)
# in the output heat map each value represents the idf values of the label word, the color represents the intersection with inputs title